# Feedback Analysis
### The feedback of the passenger from Skytrax (tigerair):
https://www.airlinequality.com/airline-reviews/tigerair-australia/page/1/

In [226]:
from bs4 import BeautifulSoup
import requests
import csv
import bs4

# Check the url address
def check_link(url):
    try:
        request = requests.get(url)
        request.raise_for_status() #  To check that a request is successful
        request.encoding = request.apparent_encoding # The apparent encoding, provided by the chardet library
        return request.text
    except:
        print("Unable to connect to the server")

def extract_review(rs):
    soup = BeautifulSoup(rs, "html.parser")
    table = soup.find_all("table")[0]
    review=[]
    
    for idx,tr in enumerate(soup.find_all('div',class_="body")): # extract the table label
        try:
            Title=None
            User=None
            Comments=None
            Title=tr.contents[1].contents[0]
            Passenger_Name=tr.contents[3].contents[1].text.strip('\n')
            Comments=tr.contents[5].contents[1].text
            #Comments=tr.contents[5].contents[1].text.split('|')[1].lstrip()
        except:
            print('error')        
        for idx, tr in enumerate(tr.find_all('div',class_="review-stats")): # extract the table labeltc_mobile
            tds = tr.find_all('td') 
            Aircrift=None
            Travel_Type=None
            Route=None
            Date_Flown=None
            Seat_Comfort=None
            Cabin_Staff_Service=None
            Ground_Service=None
            Value_For_Money=None
            Recommended=None
            for i in tds:
                if i.contents[0]=="Aircraft":
                    Aircrift=i.nextSibling.contents[0]
                elif i.contents[0]=="Type Of Traveller":
                    Travel_Type=i.nextSibling.contents[0]
                elif i.contents[0]=="Route":
                    Route=i.nextSibling.contents[0]
                elif i.contents[0]=="Date Flown":
                    Date_Flown=i.nextSibling.contents[0]
                elif i.contents[0]=="Seat Comfort":
                    Seat_Comfort=str(i.nextSibling.nextSibling.contents).count('star fill')
                elif i.contents[0]=="Cabin Staff Service":
                    Cabin_Staff_Service=str(i.nextSibling.nextSibling.contents).count('star fill')
                elif i.contents[0]=="Ground Service":
                    Ground_Service=str(i.nextSibling.nextSibling.contents).count('star fill')
                elif i.contents[0]=="Value For Money":
                    Value_For_Money=str(i.nextSibling.nextSibling.contents).count('star fill')
                elif i.contents[0]=="Recommended":
                    Recommended=i.nextSibling.contents[0]
            review.append((Aircrift,Travel_Type,Route,Date_Flown,Seat_Comfort,\
                           Cabin_Staff_Service,Ground_Service,Value_For_Money,Recommended,Title,Passenger_Name,Comments))
        print(review[-1])
        print()
    return review
            
# # Aircraft,Type Of Traveller,Route, Date Flown,Seat Comfort,Cabin Staff Service,Ground Service,Value For Money,Recommended

def main():
    
    # check the accessible
    url = "https://www.airlinequality.com/airline-reviews/tigerair-australia/"
    rs = check_link(url)
    
    # Scrapy the date from page1-page20
    start_urls = ['https://www.airlinequality.com/airline-reviews/tigerair-australia/page/' + str(i) for i in range(1,3)]
    result=[]
    for page in start_urls:
        rs = check_link(page)
        result+=extract_review(rs)
    
    # Write to the tigerair-australia save to csv
    
#     with open("tigerair-australia.csv","w",newline='',encoding='utf-8') as csvfile: 
#         writer = csv.writer(csvfile)
#         writer.writerow(["Aircrift","Travel_Type","Route","Date_Flown","Seat_Comfort",
#                            "Cabin_Staff_Service","Ground_Service","Value_For_Money","Recommended","Title",\
#                          "Passenger_Name","Comments"])
#         for row in result:
#              writer.writerow(row)

    print("Done")
            
main()

(None, 'Solo Leisure', 'Melbourne to Brisbane', 'July 2019', 1, 1, 1, 1, 'no', '"delayed 2 hrs 35 mins"', 'I Shirani', '✅ Trip Verified |  Melbourne to Brisbane delayed 2 hrs 35 mins. Unfriendly staff, Boarded half the plane when they realised there was a mechanical problem with the plane, a mechanical problem 20 minutes before it was supposed to take off, they worked on the problem for 2 hours which makes it seem I was a bit more serious than a small error. Would never fly again.')

('Boeing 737', 'Couple Leisure', 'Melbourne to Adelaide', 'July 2019', 2, 1, 2, 2, 'no', '"staff seem to be ill trained"', 'Sal Provino', "✅ Trip Verified |  Melbourne to Adelaide. My wife and I recently flew TigerAir to Melbourne and on the way back home this airline had a staff member with portable scales weigh cabin bags of every single passenger lined up to board the plane. I was 1.4kg over (I had a bottle of water in my bag) and I was ordered to march over to the service desk and pay $36.45 for the ex

In [231]:
import pandas as pd
tigerair_df=pd.read_csv('tigerair-australia.csv',dtype=str)
tigerair_df.head(5)

,Aircrift,Travel_Type,Route,Date_Flown,Seat_Comfort,Cabin_Staff_Service,Ground_Service,Value_For_Money,Recommended,Title,Passenger_Name,Comments
0,NaN,Solo Leisure,Melbourne to Brisbane,July 2019,1,1,1,1,no,"""delayed 2 hrs 35 mins""",I Shirani,✅ Trip Verified | Melbourne to Brisbane delay...
1,Boeing 737,Couple Leisure,Melbourne to Adelaide,July 2019,2,1,2,2,no,"""staff seem to be ill trained""",Sal Provino,✅ Trip Verified | Melbourne to Adelaide. My w...
2,NaN,Couple Leisure,Brisbane to Darwin,July 2019,NaN,NaN,1,1,no,"""We arrived three minutes late""",Stephanie Bandy,✅ Trip Verified | Brisbane to Darwin. Absolut...
3,NaN,Solo Leisure,Sydney to Melbourne,July 2019,4,5,5,5,yes,"""could not have been more helpful""",Amanda Coultas-Roberts,❎ Not Verified | Sydney to Melbourne. My 86 ye...
4,NaN,Business,Brisbane to Sydney,July 2019,NaN,NaN,1,1,no,"""no respect to their customers""",R Marlen,✅ Trip Verified | Brisbane to Sydney. I made ...
